In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from Bio import SeqIO
%matplotlib inline

# SCOP benchmark

In [50]:
!ls ../data/scop

scop_triples.txt


In [51]:
!ls -lhrt ../results/alignments/blast

total 128
-rw-r--r--@ 1 jmorton  1396    63K Feb  2 14:41 scop_alignments.txt


In [52]:
# Careful the file paths in this notebook may have to change according to the
# local file paths 

data_dir = '../data/scop'
fname = f'{data_dir}/scop_triples.txt'

triples = pd.read_table(fname, header=None, sep='\s+')


blast_df = pd.read_table('../results/alignments/blast/scop_alignments.txt', header=None)
blast_df.columns = [
    'cur.id', 'hit.id', 'i', 
    'qs', 'qe', 'he', 'hs', 
    'query_s', 'hit_s', 'aln_s',
    'bitscore', 'evalue'
]

In [54]:
blast_df = blast_df.set_index(['cur.id', 'hit.id'])
# drop duplicates
blast_df = blast_df.loc[~blast_df.index.duplicated(keep='first')]

In [64]:
idx1 = list(triples[[0, 1]].apply(lambda x: (x[0], x[1]), axis=1).values)
idx2 = list(triples[[0, 2]].apply(lambda x: (x[2], x[0]), axis=1).values)

In [56]:
blast_df.loc[idx1].shape

/Users/jmorton/miniconda3/envs/gert/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


(6000, 10)

In [62]:
blast_df.loc[idx1].dropna()

/Users/jmorton/miniconda3/envs/gert/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,,i,qs,qe,he,hs,query_s,hit_s,aln_s,bitscore,evalue
cur.id,hit.id,,,,,,,,,,
8028970,8020138,0.0,4.0,97.0,111.0,19.0,GVLRLMVTLSGEEVLEVVPHIGYLHTGFEKTMEHRTYLQNITYTPR...,GHLRIEVEVEGG-VIKEARSCATLFRGIETILKGRDPRDAQHFTQR...,G LR+ V + G V++ L G E ++ R +T R...,27.7202,4.448130e+00
8024875,8027318,0.0,41.0,227.0,253.0,68.0,SQVKKMGELGLLAMDVPEELSGAGLDYLAYSIALEEISRGCASTGV...,TMVKKMREYGI---SDPEEIMWFKKLYLANFVE-------------...,+ VKKM E G+ PEE+ YLA + ...,63.5438,1.346390e-12
8021879,8021923,0.0,10.0,134.0,133.0,6.0,IMVGPGTGIAPFMGFIQERAWLRE--QGKEVGETLLYYGCRRSDED...,IMIATGTGVAPFRGYLR-RMFMEDVPNYRFGGLAWLFLGVANSDS-...,IM+ GTG+APF G+++ R ++ + + G L+ G SD ...,55.8398,6.399250e-11
8058589,8003702,0.0,173.0,351.0,371.0,194.0,LQPMVSVVTNMEPDHMDTYEGDFEKMKATYVKFLHNLPFYGLAVMC...,LKFAASVFTNLSRDHLD-YHGDMEHYEAAK-WLLYSEHHCGQAIIN...,L+ SV TN+ DH+D Y GD E +A L++ G A++ ...,33.1130,1.049390e-01
8023360,8026077,0.0,51.0,109.0,69.0,13.0,LDNLVYRLGLARTRRQARQLVTHGHIIVDGSRVNIPSYRVKPGQTI...,LCDLLKLEGWSESGAQAKIAIAEGQVKVDGAVETRKRCKIVAGQTV...,L +L+ G + + QA+ + G + VDG+ ++ GQT+...,23.4830,8.215460e+00
...,...,...,...,...,...,...,...,...,...,...,...
8047851,8023589,0.0,94.0,162.0,149.0,74.0,GDVIEVHGKH----EERQDEHGFISR---EFHRKYRIPADVDPLTI...,GNVLVVSGERTKEKEDKNDKWHRVERSSGKFVRRFRLLEDAKVEEV...,G+V+ V G+ E++ D+ + R +F R++R+ D +...,29.6462,1.916100e-01
8045414,8048744,0.0,150.0,192.0,403.0,360.0,FGIVAWAILYFIRVPKEEELLIEEFGDEYIEYMGKTG-RLFPK,FHIVYFTILLLHREKRDDAMCLAKYGEDWLQYRKKVPWRIVPK,F IV + IL R +++ + + ++G+++++Y K R+ PK,25.4090,8.485260e+00
8027318,8024875,0.0,68.0,253.0,227.0,41.0,TMVKKMREYGI---SDPEEIMWFKKLYLANFVE-------------...,SQVKKMGELGLLAMDVPEELSGAGLDYLAYSIALEEISRGCASTGV...,+ VKKM E G+ PEE+ YLA + ...,63.5438,1.556210e-12


In [65]:
blast_df.loc[idx2].dropna()

/Users/jmorton/miniconda3/envs/gert/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,,i,qs,qe,he,hs,query_s,hit_s,aln_s,bitscore,evalue
cur.id,hit.id,,,,,,,,,,
8021750,8027576,0.0,56.0,95.0,82.0,41.0,DMNGCFNDGAQAATG--CTYGKGLFSLLGYGKLALILYRPG,DLNAPLNDSEQAQLTRLLQYGPALSSHTPAGKLLLVTPRPG,D+N ND QA YG L S GKL L+ RPG,29.6462,0.193596
8030427,8021924,0.0,55.0,105.0,49.0,1.0,TLIVESEELSKVDNQIGASIGKIIEILQGLNETSTNAYRTLPINNM...,TLKLLRKEIDKIDNQIISLLKKRLEIAQAIGKIKKEL--NLPIEDR...,TL + +E+ K+DNQI + + K +EI Q + + LPI + ...,25.0238,8.459270
8028962,8027279,0.0,3.0,52.0,338.0,288.0,DPRFERTLYAHVGK--EGSWTLDYYLRHGGYETAKRVLKEKTPDEV...,DPRFAGTIKITTSAVIDNRANLNYLLSHSGLDYKRNILNDRNP-VV...,DPRF T+ + L+Y L H G + + +L ++ P V...,26.1794,5.807930


In [66]:
triples.shape

(6000, 3)

In [67]:
266 / 6000

0.044333333333333336

# PFAM benchmark

In [120]:
!ls -lhrt ../results/elmo_epoch5_distances.txt

-rw-r--r--@ 1 jmorton  1396   132K Feb  2 18:07 ../results/elmo_epoch5_distances.txt


In [134]:
data_dir = '../data/pfam'
fname = f'{data_dir}/pfam_benchmark_pairs_50k.txt'

triples = pd.read_table(fname, header=None, sep='\s+')

elmo = pd.read_table('../results/elmo_epoch5_distances.txt', sep='\s+', header=None)

In [132]:
elmo = elmo.set_index([0, 1, 2])
idx = elmo.index

In [136]:
triples = triples.set_index([0, 1, 2])
triples = triples.loc[idx]
triples = triples.reset_index()

In [137]:
blast_df = pd.read_table(f'{data_dir}/pfam_scores.txt', header=None)
blast_df.columns = [
    'cur.id', 'hit.id', 'i', 
    'qs', 'qe', 'he', 'hs', 
    'query_s', 'hit_s', 'aln_s',
    'bitscore', 'evalue'
]

In [138]:
blast_df = blast_df.set_index(['cur.id', 'hit.id'])
# drop duplicates
#blast_df = blast_df.loc[~blast_df.index.duplicated(keep='first')]

In [139]:
idx1 = list(triples[[0, 1]].apply(lambda x: (x[0], x[1]), axis=1).values)
idx2 = list(triples[[0, 2]].apply(lambda x: (x[0], x[2]), axis=1).values)
idx1r = list(triples[[0, 1]].apply(lambda x: (x[1], x[0]), axis=1).values)

In [140]:
blast_df.loc[idx1].dropna()

/Users/jmorton/miniconda3/envs/gert/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,,i,qs,qe,he,hs,query_s,hit_s,aln_s,bitscore,evalue
cur.id,hit.id,,,,,,,,,,
Q9Z6Y2,Q8KA15,0.0,12.0,743.0,718.0,3.0,AEVIVGSNINKVLDYGVPENLEHITKGTAVTISLRGGKKVGVIYQI...,VKVVLPLPIRKYFKYFMPDSMCPII-GGRIVVPFRSKDIVGIV---...,+V++ I K Y +P+++ I G + + R VG++ ...,157.1470,1.333540e-39
P84903,P70302,0.0,0.0,685.0,685.0,0.0,MDVCARLALWLLWGLLLHQGQSLSHSHSEKNTGASSGATSEESTEA...,MDVCARLALWLLWGLLLHQGQSLSHSHSEKNTGASSGATSEESTEA...,MDVCARLALWLLWGLLLHQGQSLSHSHSEKNTGASSGATSEESTEA...,1393.2500,0.000000e+00
P74435,B9E150,0.0,2.0,197.0,184.0,1.0,MEVKICGLRHPAQAQAIAALGFTTLGFICVEASSRYVSSREIELVL...,IKVKICGLKRDEDIKCVNRYKPDYVGFV-FSKSKRQVNLEQAKMLI...,++VKICGL+ + + +GF+ S R V+ + ++++...,80.4925,1.573020e-18
Q9U2C4,Q7Z7M9,0.0,69.0,437.0,799.0,430.0,APRE--GPGEKGKPVVLT-GKDAELGQADMKKWFMNVHASDKISLD...,SPRDPKAPGQFGRPVVVPHGKEKE-AERRWKEGNFNVYLSDLIPVD...,+PR+ PG+ G+PVV+ GK+ E + K+ NV+ SD I +D...,353.2140,9.416830e-111
P39205,Q5HLX6,0.0,216.0,586.0,402.0,34.0,NLNKILLE-MNAPVNIPPFRASIKDGYAMKS---TGFSGTKR----...,SLGHILAEDIVATYDIPRFNKSPYDGFAIRSEDSQGASGENRIEFE...,+L IL E + A +IP F S DG+A++S G SG R ...,121.7090,1.496010e-29
...,...,...,...,...,...,...,...,...,...,...,...
P77161,Q49A26,0.0,1.0,283.0,551.0,268.0,KLGFIGLGIMGTPMAINLARAGHQLHVTT-IGPVADELLSLGAVSV...,KIGFLGLGLMGSGIVSNLLKMGHTVTVWNRTAEKCDLFIQEGARLG...,K+GF+GLG+MG+ + NL + GH + V D + GA ...,110.5380,5.914190e-27
P32701,P76446,0.0,238.0,518.0,512.0,236.0,FYNHLIFALPAGILGSLVLLLLWLRIRQNYLSPKRKLQRALEKHQL...,WYAFLLGGMSGTVVGLLCYYLMSVRMR-----PGREIMTAIKREQF...,+Y L+ + ++G L L+ +R+R P R++ A+++ Q ...,152.5250,5.857170e-40
Q8K209,Q8K3V3,0.0,0.0,687.0,687.0,0.0,MAVQVLRQMVYFLLSLFSLVQGAHSGSPREDFRFCGQRNQTQQSTL...,MAVQVLLQMVYFLLTLLFLVQGAHGASPREDFRFCGQRNQTQQSTL...,MAVQVL QMVYFLL+L LVQGAH SPREDFRFCGQRNQTQQSTL...,1240.7100,0.000000e+00


In [145]:
elmo.shape

(1373, 7)

In [146]:
1365 / 1373

0.9941733430444283